In [4]:
#Read the data for January. How many columns are there?
import pandas as pd

df = pd.read_parquet('/content/yellow_tripdata_2025-01.parquet')
len(df.columns)


20

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3475226 entries, 0 to 3475225
Data columns (total 20 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int32         
 1   tpep_pickup_datetime   datetime64[us]
 2   tpep_dropoff_datetime  datetime64[us]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int32         
 8   DOLocationID           int32         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  Airport_fee           

In [6]:
# let's compute the duration variable. It should contain the duration of a ride in minutes.
df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

In [7]:
#What's the standard deviation of the trips duration in January?
df.duration.std()

38.71358219513072

In [11]:
# we need to check the distribution of the duration variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive)
df_ = df[(df.duration >= 1) & (df.duration <= 60)]


In [12]:
#What fraction of the records left after you dropped the outliers?
len(df_) / len(df)


1.0

In [14]:
#lets apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.

#Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will label encode them)
#Fit a dictionary vectorizer
#Get a feature matrix from it
from sklearn.feature_extraction import DictVectorizer

categorical = ['PULocationID', 'DOLocationID']
df_[categorical] = df_[categorical].astype(str)
train_dicts = df_[categorical].to_dict(orient='records')

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)



In [15]:
#What's the dimensionality of this matrix (number of columns)?
X_train.shape[1]


519

In [ ]:
# Train a plain linear regression model with default parameters, where duration is the response variable
# Calculate the RMSE of the model on the training data
# What's the RMSE on train?
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

y_train = df_['duration']

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)


In [20]:
# What's the RMSE on train?
res = mean_squared_error(y_train, y_pred)
res ** 0.5

7.707822122519484

In [22]:
# Now let's apply this model to the validation dataset (February 2023).
# What's the RMSE on validation?
df_val = pd.read_parquet('/content/yellow_tripdata_2025-02.parquet')
df_val['duration'] = df_val.tpep_dropoff_datetime - df_val.tpep_pickup_datetime
df_val.duration = df_val.duration.apply(lambda td: td.total_seconds() / 60)
df_val = df_val[(df_val.duration >= 1) & (df_val.duration <= 60)]
val_dicts = df_val[categorical].to_dict(orient='records')
X_val = dv.transform(val_dicts)
y_val = df_val['duration']
y_pred = lr.predict(X_val)
res = mean_squared_error(y_val, y_pred)
res ** 0.5




12.581892094904086